<a href="https://colab.research.google.com/github/AnshuMishra01/Ensemble-based-hotel-recommender-system/blob/main/RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df=pd.read_csv("/content/review.csv",encoding='ISO-8859–1',header=0)
df.head()

,ï»¿text,Unnamed: 1,Sentimental_1,Score,Neutral,Sentimental
0,"This hotel was nice and quiet. Did not know, t...",NaN,positive,0.69,False,positive
1,We stayed in the king suite with the separatio...,NaN,negative,0.01,False,negative
2,"Parking was horrible, somebody ran into my ren...",NaN,negative,0.07,False,negative
3,Not cheap but excellent location. Price is som...,NaN,positive,0.93,False,positive
4,If you get the room that they advertised on th...,NaN,negative,0.39,False,negative


In [ ]:
df.rename(columns={df.columns[0]: 'text',df.columns[1]: "lol", df.columns[5]: 'sentimental'}, inplace=True)

In [ ]:
df1= df.drop(['lol','Sentimental_1','Score','Neutral'], axis=1)
df1.head()

,text,sentimental
0,"This hotel was nice and quiet. Did not know, t...",positive
1,We stayed in the king suite with the separatio...,negative
2,"Parking was horrible, somebody ran into my ren...",negative
3,Not cheap but excellent location. Price is som...,positive
4,If you get the room that they advertised on th...,negative


In [ ]:
df1.loc[ df1['sentimental'] =='positive', 'sentimental'] = 2
df1.loc[ df1['sentimental'] =='negative', 'sentimental'] = 0
df1.loc[ df1['sentimental'] =='neutral', 'sentimental'] = 1

In [ ]:
df1.head()

,text,sentimental
0,"This hotel was nice and quiet. Did not know, t...",2
1,We stayed in the king suite with the separatio...,0
2,"Parking was horrible, somebody ran into my ren...",0
3,Not cheap but excellent location. Price is som...,2
4,If you get the room that they advertised on th...,0


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
df1.head()

,text,sentimental
0,"This hotel was nice and quiet. Did not know, t...",2
1,We stayed in the king suite with the separatio...,0
2,"Parking was horrible, somebody ran into my ren...",0
3,Not cheap but excellent location. Price is som...,2
4,If you get the room that they advertised on th...,0


In [ ]:
# return the wordnet object value corresponding to the POS tag
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)


In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
df1["text_clean"] = df1["text"].apply(lambda x: clean_text(x) inplace=True)
df1.head()

,text,sentimental,text_clean
0,"This hotel was nice and quiet. Did not know, t...",2,hotel nice quiet know train track near train p...
1,We stayed in the king suite with the separatio...,0,stay king suite separation bedroom live space ...
2,"Parking was horrible, somebody ran into my ren...",0,park horrible somebody run rental car stay get...
3,Not cheap but excellent location. Price is som...,2,cheap excellent location price somewhat standa...
4,If you get the room that they advertised on th...,0,get room advertised website pay may lucky.if s...


In [ ]:

import pandas as pd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Assuming df is your pandas DataFrame containing text data in the column "text_clean"

# Prepare TaggedDocument objects
documents = [TaggedDocument(words=text.split(), tags=[str(i)]) for i, text in enumerate(df1["text_clean"])]

# Train a Doc2Vec model with our text data
model = Doc2Vec(vector_size=5, window=2, min_count=1, workers=4)
model.build_vocab(documents)
model.train(documents, total_examples=model.corpus_count, epochs=10)  # Adjust epochs as needed

# Transform each document into a vector
doc2vec_df = pd.DataFrame([model.infer_vector(text.split()) for text in df1["text_clean"]], columns=["doc2vec_vector_" + str(i) for i in range(5)])

# Concatenate the vectors with the original DataFrame
df1 = pd.concat([df1, doc2vec_df], axis=1, inplace=True)


In [ ]:
# add number of characters column
df1["nb_chars"] = df1["text"].apply(lambda x: len(x))

# add number of words column
df1["nb_words"] = df1["text"].apply(lambda x: len(x.split(" ")))

In [ ]:
import sklearn
print(sklearn.__version__)

1.4.0


In [ ]:
# # add tf-idfs columns
# from sklearn.feature_extraction.text import TfidfVectorizer
# tfidf = TfidfVectorizer(min_df = 10)
# tfidf_result = tfidf.fit_transform(df1["text_clean"]).toarray()
# tfidf_df = pd.DataFrame(tfidf_result, columns = tfidf.get_feature_names())
# tfidf_df.columns = ["word_" + str(x) for x in tfidf_df.columns]
# tfidf_df.index = df1.index
# df1 = pd.concat([df1, tfidf_df], axis=1)
# from sklearn.feature_extraction.text import CountVectorizer

# countvec = CountVectorizer(min_df=10)
# countvec_result = countvec.fit_transform(df1["text_clean"]).toarray()
# countvec_df = pd.DataFrame(countvec_result, columns=countvec.get_feature_names_out())
# countvec_df.columns = ["word_" + str(x) for x in countvec_df.columns]
# countvec_df.index = df1.index
# df1 = pd.concat([df1, countvec_df], axis=1)

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(min_df=10)
tfidf_result = tfidf.fit_transform(df1['text_clean']).toarray()
tfidf_df = pd.DataFrame(tfidf_result, columns=tfidf.get_feature_names_out())
tfidf_df.columns = ["word_" + str(x) for x in tfidf_df.columns]
tfidf_df.index = df1.index
df1 = pd.concat([df1, tfidf_df], axis=1)

In [ ]:
df1.columns

Index(['text', 'sentimental', 'text_clean', 'doc2vec_vector_0',
       'doc2vec_vector_1', 'doc2vec_vector_2', 'doc2vec_vector_3',
       'doc2vec_vector_4', 'nb_chars', 'nb_words',
       ...
       'word_young', 'word_your', 'word_youre', 'word_yours', 'word_yr',
       'word_yummy', 'word_zaza', 'word_zephyr', 'word_zero', 'word_zoo'],
      dtype='object', length=3569)

In [ ]:
!pip install scikit-learn==1.4.0

In [ ]:
label = "sentimental"
ignore_cols = [label, "text", "text_clean"]
features = [c for c in df1.columns if c not in ignore_cols]

In [ ]:
!pip install numpy scipy

In [ ]:
!pip show scikit-learn

Name: scikit-learn
Version: 1.4.1.post1
Summary: A set of python modules for machine learning and data mining
Home-page: https://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: bigframes, fastai, imbalanced-learn, librosa, mlxtend, qudida, sklearn-pandas, yellowbrick


In [ ]:
dfx = df1.drop(["sentimental", "text", "text_clean"], axis=1)
dfx.head()

,doc2vec_vector_0,doc2vec_vector_1,doc2vec_vector_2,doc2vec_vector_3,doc2vec_vector_4,nb_chars,nb_words,word_aaa,word_aarp,word_ability,...,word_young,word_your,word_youre,word_yours,word_yr,word_yummy,word_zaza,word_zephyr,word_zero,word_zoo
0,0.274186,0.134787,-0.082600,0.067342,-0.053120,214,39,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.221172,0.038480,0.333654,-0.098670,-0.118380,325,61,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.177926,0.296083,0.065797,-0.247428,-0.215601,288,54,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.324708,0.153940,0.181494,0.009597,-0.580914,310,52,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.691657,0.128422,0.197661,-0.176713,-0.646117,331,61,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Verify the shape of X_train and y_train
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)

Shape of X_train: (8000, 3566)
Shape of y_train: (8000,)


In [ ]:
assert X_train.shape[0] == y_train.shape[0]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(dfx, df1["sentimental"], test_size = 0.20, random_state = 42)

In [ ]:
X_train.shape

(8000, 3566)

In [ ]:
X_test.shape

(2000, 3566)

In [ ]:
y_train.shape

(8000,)

In [ ]:
y_test.shape

(2000,)

In [ ]:
print(y_train.dtype)

object


In [ ]:
y_train.describe()

count     8000
unique       3
top          2
freq      4703
Name: sentimental, dtype: int64

In [ ]:
y_train = y_train.astype(str)

In [ ]:
# train a random forest classifier
rf = RandomForestClassifier(n_estimators = 100, random_state = 42)
rf.fit(X_train, y_train)

# show feature importance
feature_importances_df = pd.DataFrame({"feature": features, "importance": rf.feature_importances_}).sort_values("importance", ascending = False)
feature_importances_df.head(20)

,feature,importance
4,doc2vec_vector_4,0.011442
0,doc2vec_vector_0,0.011130
2,doc2vec_vector_2,0.011016
3,doc2vec_vector_3,0.010555
1,doc2vec_vector_1,0.010250
5,nb_chars,0.009971
6,nb_words,0.008110
1521,word_hotel,0.006511
2666,word_room,0.006460
2975,word_stay,0.006190


In [ ]:
print(f"y_test data type: {type(y_test)}")
print(f"y_pred data type: {type(y_pred)}")

y_test data type: <class 'pandas.core.series.Series'>
y_pred data type: <class 'pandas.core.series.Series'>


In [ ]:
print(f"Unique values in y_test: {y_test.unique()}")

Unique values in y_test: [2 0 1]


In [ ]:
y_test = y_test.astype(int)

In [ ]:
print(f"Model type: {type(rf)}")

Model type: <class 'sklearn.ensemble._forest.RandomForestClassifier'>


In [ ]:
# prompt: write the code to test my model

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = rf.predict(X_test)

y_test = y_test.astype(int)
y_pred = y_pred.astype(int)

print("Accuracy score:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy score: 0.5955

Classification Report:
              precision    recall  f1-score   support

           0       0.52      0.19      0.28       797
           1       0.00      0.00      0.00        28
           2       0.61      0.88      0.72      1175

    accuracy                           0.60      2000
   macro avg       0.38      0.36      0.33      2000
weighted avg       0.56      0.60      0.54      2000


Confusion Matrix:
[[ 153    0  644]
 [   5    0   23]
 [ 137    0 1038]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  # set those with 0 denominator to `zero_division`, and 0 when "warn"
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  # set those with 0 denominator to `zero_division`, and 0 when "warn"
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  # set those with 0 denominator to `zero_division`, and 0 when "warn"
